In [1]:
import sklearn
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train2 = train.copy()
test2 = test.copy()

train2.drop(train2[(train2.Age <= 0) & (train2.Age >= 100)].index, inplace=True)
test2.drop(test2[(test2.Age <= 0) & (test2.Age >= 100)].index, inplace=True)


# rimuovo i valori mancanti
train2.replace("", np.nan, inplace=True)
train2.replace(" ", np.nan, inplace=True)
train2.replace("?", np.nan, inplace=True)
test2.replace("", np.nan, inplace=True)
test2.replace(" ", np.nan, inplace=True)
test2.replace("?", np.nan, inplace=True)

train2['ScheduledDay'] = pd.to_datetime(train2['ScheduledDay'])
test2['ScheduledDay'] = pd.to_datetime(test2['ScheduledDay']) 
train2['Period'] = pd.cut(train2.ScheduledDay.dt.hour,[0,8,12,18,24],labels=['Night','Morning','Afternoon','Evening'])
test2['Period'] = pd.cut(test2.ScheduledDay.dt.hour,[0,8,12,18,24],labels=['Night','Morning','Afternoon','Evening'])

features = ['Gender', 'Age', 'Neighbourhood', 'AppointmentDay' ,'Scholarship', 'Hipertension',
       'Diabetes', 'Alcoholism', 'Handcap', 'SMS_received','Period']

for elemento in train2:
  if elemento=="Age" or elemento=="Scholarship" or elemento == "Hipertension" or elemento == "Diabetes" or elemento == "Alcoholism" or elemento == "Handcap" or elemento == "SMS_received":
   train2[elemento] = train2[elemento].astype('float')
  else:
   train2[elemento] = (pd.Categorical(train2[elemento])).codes.astype('float')
for elemento in test2:
  if elemento=="Age" or elemento=="Scholarship" or elemento == "Hipertension" or elemento == "Diabetes" or elemento == "Alcoholism" or elemento == "Handcap" or elemento == "SMS_received":
   test2[elemento] = test2[elemento].astype('float')
  else:
   test2[elemento] = (pd.Categorical(test2[elemento])).codes.astype('float')

X=train2[features]
Y=train2['No-show']
X_test=test2[features]
Y_test=test2['No-show']


label_encoder = LabelEncoder()

Y = label_encoder.fit_transform(Y)
Y_test = label_encoder.transform(Y_test)

In [4]:
pipe = Pipeline([
        ('scaler', StandardScaler()),
        ('reduce_dim', PCA()),
        ('classifier', RandomForestClassifier(random_state=42) )
        ])

# RandomForest è un algoritmo che (a differenza del DecisionTree) genera molteplici alberi. 
# Ogni albero classifica un oggetto basandosi sui suoi attributi ed esprimerà quindi un voto per quella classe. 
# Nel caso del RandomForestClassifier si prenderà quindi in considerazione la classificazione che ha ricevuto più voti fra tutti gli altri alberi nella foresta.

pipe=pipe.fit(X,Y)
print('Score finale di test: ', pipe.score(X_test, Y_test))


train_predictions = pipe.predict(X)
train_acc=metrics.accuracy_score(Y,train_predictions)
print('Accuratezza train: %s'  % train_acc)

test_predictions = pipe.predict(X_test)
test_acc=metrics.accuracy_score(Y_test,test_predictions)
print('Accuratezza test: %s'  % test_acc)

print('%s%%' % int(test_acc*100))


Score finale di test:  0.7737065309584393
Accuratezza train: 0.9865273693734449
Accuratezza test: 0.7737065309584393
77%
